In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import lyricsgenius as lg
import os
from fuzzywuzzy import fuzz

client_id = 'accd0aa479164ddcbf1cbf822512b80b'
client_secret = '58bfc467435045e7b61c86fb03385729'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
spot = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

genius_client_id = '6ik7FFRHyv94azwRfLlDIpTAS-710CM1BoWDfO2x82LkpIz9g9Oo5-YQbp_B_xzA'
genius_client_secret = '23fPMx4U0q6TJRT0ss3aIpXRhT9w0taLUf-3_w2iRxAYU1b3uKSPvucwdqbkf8iObaqM5-_f0RXOTV92bLeLvQ'

genius = lg.Genius('SFSvR6r7o3-3g123P0DUAFkiqsvBZaptEbjQw2Nw6rAPLUkUA9d89_8VdNa9tK50')

# Keeping track of finished artists

artists = [
           'Drake', 'Travis Scott','Juice WRLD', 'J. Cole', 'Lil Baby', 'DaBaby'
]
          # 'Lil Uzi Vert','Kendrick Lamar', 'Roddy Ricch', 'Meek Mill','Kanye West', 
          # 'JAY-Z', 'Isaiah Rashad', 'Pusha T', 'Earl Sweatshirt', 'Tyler, The Creator',
          # 'ASAP Rocky', 'Playboi Carti', 'Future', 'Young Thug', 'Chance the Rapper', 
          # 'Schoolboy Q', '21 Savage', 'Lil Wayne','Eminem', 'Pop Smoke', 
          # 'Lil Durk', 'A Boogie wit da Hoodie', 'Gunna', 'Migos', 
          # 'Trippie Redd', 'Kid Cudi', 'Big Sean' 
#]

In [3]:
# Cleans title name
def clean_title(song_name):
    cleaned = song_name.replace('\'','')
    cleaned = cleaned.replace('’','')
    return cleaned

def get_fuzz_ratio(var1, var2):
    return fuzz.ratio(var1, var2)

# For all artists, gather their data into one dataframe
# returns this dataframe
def call_data(artists):
    #slice = artists[0:2]
    # initialize lists to be added onto extracted object
    tracks, views, popularity_index, artist_name = ([] for i in range(4) )
    for artist in artists:
        print(f"Working on {artist}")
        check = 0
        while check != 1:
            try:
                count = 1 # Will serve as Popularity Index 
                song_list = (genius.search_artist(artist, max_songs=10,get_full_info=False)).songs
                for song in song_list:
                    #DEBUG print(f'Song {count}: {song.title}')
                    artist_name.append(artist)
                    tracks.append( clean_title(song.title) )
                    popularity_index.append(count)
                    try:
                        views.append( song.stats.pageviews )
                    except:
                        views.append(0)
                    count += 1
    # Creation of DataFrame
                extracted_songs = pd.DataFrame()
                extracted_songs['Song'] = tracks
                extracted_songs['Artist'] = artist_name
                extracted_songs['PopIndex'] = popularity_index
                extracted_songs['PageViews'] = views
                check = 1
                print(f"Ending song extraction for {artist}")
            except:
                continue
            break
    return extracted_songs

In [ ]:
df = call_data(artists)

Working on Drake
Searching for songs by Drake...

Searching for songs by Drake...

Searching for songs by Drake...

Searching for songs by Drake...

Song 1: "God’s Plan"


In [79]:
main = df
spot_names = []
spot_artists = []
ids = []
problem_songs = []
for i in main.index:
### only take song if fuzz_song > 75 and fuzz_artist > 80
    artist = main.loc[i]['Artist']
    song = main.loc[i]['Song']
    spot_name = None
    song_artist = None
    #DEBUGGING print(f"searching {song} {artist}")
    
    #Search Spotify for song + artist
    search_count = 0
    found = 0
    while( found < 1 and search_count < 3):
        track = spot.search(q=str(song + ' ' + artist))
        try: 
            spot_name = track['tracks']['items'][search_count]['name']
            song_artist = track['tracks']['items'][search_count]['album']['artists'][0]['name']
        except:
            print(f"{song} NOT SEARCHED SUCCESSFULLY at track var")
            problem_songs.append(f"{song} by {artist}")
            spot_names.append('NA')
            spot_artists.append('NA')
            ids.append('NA')
            found = 1
        fuzz_song = get_fuzz_ratio(song, spot_name)
        fuzz_artist = get_fuzz_ratio(artist, song_artist)
        fuzz_partial = fuzz.partial_ratio(song, spot_name)
        # take higher number of the 2 song fuzzes
        song_fuzz = (fuzz_song + fuzz_partial) / 2
        #print(f"Fuzz artist: {fuzz_artist} AND Fuzz song: {song_fuzz}")
        #print(f"{spot_name} by {song_artist} in SPOTIFY")
        if ( fuzz_artist > 75 and song_fuzz > 50):
            found = 1
            spot_artists.append(song_artist)
            spot_names.append(clean_title(spot_name))
            song_id = track['tracks']['items'][0]['id']
            ids.append(song_id)
            search_count = 3
        elif(found < 1 and search_count == 2):
            print(f"{song} by {artist} not found.")
            problem_songs.append(f"{song} by {artist}")
            spot_names.append('NA')
            spot_artists.append('NA')
            ids.append('NA')
        search_count +=1


Drugs You Should Try It by Travis Scott not found.
Skyfall by Travis Scott not found.
​moonlight by Juice WRLD not found.
Unreleased Songs [Discography List] by Juice WRLD not found.
Autograph (on my line) by Juice WRLD not found.
RUN by Juice WRLD not found.
Hide by Juice WRLD not found.
Beatbox Freestyle by DaBaby not found.
DaBaby Freestyle | LA Leakers Freestyle #076 by DaBaby not found.
4X by DaBaby not found.
Buy It NOT SEARCHED SUCCESSFULLY at track var
Free Uzi by Lil Uzi Vert not found.
Ricch Forever by Roddy Ricch not found.
Fucc It Up by Roddy Ricch not found.
Chase tha Bag by Roddy Ricch not found.
Ricch Vibes by Roddy Ricch not found.
Wanna Know by Meek Mill not found.
War Pain by Meek Mill not found.
Open Letter by JAY-Z not found.
The Story of Adidon NOT SEARCHED SUCCESSFULLY at track var
Exodus 23:1 by Pusha T not found.
Trust You by Pusha T not found.
Blocka NOT SEARCHED SUCCESSFULLY at track var
Earl by Earl Sweatshirt not found.
Couch by Earl Sweatshirt not found.
‌s

In [81]:
problem_songs

['Drugs You Should Try It by Travis Scott',
 'Skyfall by Travis Scott',
 '\u200bmoonlight by Juice WRLD',
 'Unreleased Songs [Discography List] by Juice WRLD',
 'Autograph (on my line) by Juice WRLD',
 'RUN by Juice WRLD',
 'Hide by Juice WRLD',
 'Beatbox Freestyle by DaBaby',
 'DaBaby Freestyle | LA Leakers Freestyle #076 by DaBaby',
 '4X by DaBaby',
 'Buy It by Lil Uzi Vert',
 'Free Uzi by Lil Uzi Vert',
 'Ricch Forever by Roddy Ricch',
 'Fucc It Up by Roddy Ricch',
 'Chase tha Bag by Roddy Ricch',
 'Ricch Vibes by Roddy Ricch',
 'Wanna Know by Meek Mill',
 'War Pain by Meek Mill',
 'Open Letter by JAY-Z',
 'The Story of Adidon by Pusha T',
 'Exodus 23:1 by Pusha T',
 'Trust You by Pusha T',
 'Blocka by Pusha T',
 'Earl by Earl Sweatshirt',
 'Couch by Earl Sweatshirt',
 '\u200csolace by Earl Sweatshirt',
 'Luper by Earl Sweatshirt',
 '\u200bepaR by Earl Sweatshirt',
 'Kill by Earl Sweatshirt',
 'Drop by Earl Sweatshirt',
 'Pigions by Earl Sweatshirt',
 'Bastard by Tyler, The Creator'

In [82]:
spot_data = pd.DataFrame()
spot_data['Spot Song'] = spot_names
spot_data['Spot Artist'] = spot_artists
spot_data['Real Artist'] = main['Artist']
spot_data['Real Song'] = main['Song']
spot_data['Song ID'] = ids

In [83]:
fuzz_ratio_song = []
fuzz_ratio_artist = []
fuzz_ratio_partial_song = []

for i in range(0,len(main)):
    real_song= spot_data.loc[i]['Real Song']
    found_song = spot_data.loc[i]['Spot Song'] 
    fuzz_ratio_song.append(fuzz.ratio(real_song, found_song))
    fuzz_ratio_partial_song.append(fuzz.partial_ratio(real_song, found_song))

for i in range(0,len(main)):
    real_name= spot_data.loc[i]['Real Artist']
    found_name = spot_data.loc[i]['Spot Artist'] 
    fuzz_ratio_artist.append(fuzz.ratio(real_name, found_name))
    
spot_data['fuzz_song'] = fuzz_ratio_song
spot_data['fuzz_artist'] = fuzz_ratio_artist
spot_data['fuzz_partial_song'] = fuzz_ratio_partial_song

In [84]:
len(spot_data[spot_data['Spot Song'] == 'NA'])

57

In [85]:
spot_data[0:40]

,Spot Song,Spot Artist,Real Artist,Real Song,Song ID,fuzz_song,fuzz_artist,fuzz_partial_song
0,Gods Plan,Drake,Drake,Gods Plan,6DCZcSspjsKoFjzjrWoCdn,100,100,100
1,In My Feelings,Drake,Drake,In My Feelings,2G7V7zsVDxg1yRsu7Ew9RJ,100,100,100
2,Hotline Bling,Drake,Drake,Hotline Bling,0wwPcA6wtMf6HUMpIRdeP7,100,100,100
3,One Dance,Drake,Drake,One Dance,1zi7xx7UVEFkmKfv06H8x0,100,100,100
4,"Hold On, Were Going Home",Drake,Drake,"Hold On, Were Going Home",2aSC2xhRxOLiiZZVjhbylH,100,100,100
5,Know Yourself,Drake,Drake,Know Yourself,5InOp6q2vvx0fShv3bzFLZ,100,100,100
6,Back To Back,Drake,Drake,Back to Back,5lFDtgWsjRJu8fPOAyJIAK,92,100,92
7,All Me,Drake,Drake,All Me,7y911sqgHGHcyU4ivapjwi,100,100,100
8,From Time,Drake,Drake,From Time,0GaBIpyHvytM1UBYmqXu08,100,100,100
9,Fake Love,Drake,Drake,Fake Love,343YBumqHu19cGoGARUTsd,100,100,100


In [86]:
spot_data[40:80]

,Spot Song,Spot Artist,Real Artist,Real Song,Song ID,fuzz_song,fuzz_artist,fuzz_partial_song
40,through the late night,Travis Scott,Travis Scott,​through the late night,1Ci4wASMY4xtKVMeHA6Sd5,98,100,100
41,STOP TRYING TO BE GOD,Travis Scott,Travis Scott,STOP TRYING TO BE GOD,1xYpVRspgaLxdJLOTQkTjg,100,100,100
42,NA,NA,Travis Scott,Drugs You Should Try It,NA,0,0,0
43,NO BYSTANDERS,Travis Scott,Travis Scott,NO BYSTANDERS,6OaVWaGfhXn70ZJfdVcabn,100,100,100
44,Maria Im Drunk (feat. Justin Bieber & Young Thug),Travis Scott,Travis Scott,Maria Im Drunk,6Yqmv7XJLCrQEauMbPGZSw,44,100,100
45,3500 (feat. Future & 2 Chainz),Travis Scott,Travis Scott,3500,1SGt65i9AnXYdDQt1AtDRH,24,100,100
46,WAKE UP,Travis Scott,Travis Scott,WAKE UP,20MuVazoNMv6xjKPnRFOxG,100,100,100
47,the ends,Travis Scott,Travis Scott,​the ends,33zbOXJSwW7uf4VXXWow2O,94,100,100
48,CAROUSEL,Travis Scott,Travis Scott,CAROUSEL,7KZ5MMVgBVox9ycroB2UrI,100,100,100
49,FRANCHISE (feat. Young Thug & M.I.A.),Travis Scott,Travis Scott,FRANCHISE,4jVBIpuOiMj1crqd8LoCrJ,39,100,100


In [87]:
spot_data[80:120]

,Spot Song,Spot Artist,Real Artist,Real Song,Song ID,fuzz_song,fuzz_artist,fuzz_partial_song
80,Stay High,Juice WRLD,Juice WRLD,Stay High,21CkzgKCBCq5V4XKjiGSIj,100,100,100
81,Already Dead,Juice WRLD,Juice WRLD,Already Dead,1hB3M3POeKMLxcEFEvPeqU,100,100,100
82,Hear Me Calling,Juice WRLD,Juice WRLD,Hear Me Calling,2MFFrpPKrGnyJuVjcfdm4w,100,100,100
83,Man Of The Year,Juice WRLD,Juice WRLD,Man of the Year,7AvprzMsRJpybaalckaT8c,87,100,87
84,NA,NA,Juice WRLD,Unreleased Songs [Discography List],NA,0,0,0
85,Flaws And Sins,Juice WRLD,Juice WRLD,Flaws and Sins,0flSGzqHO12p9fU9kqaHGE,93,100,93
86,NA,NA,Juice WRLD,Autograph (on my line),NA,8,0,0
87,NA,NA,Juice WRLD,RUN,NA,40,0,50
88,Scared Of Love,Juice WRLD,Juice WRLD,Scared of Love,49QVOdWgGagzLrThkIqtK6,93,100,93
89,NA,NA,Juice WRLD,Hide,NA,0,0,0


In [88]:
spot_data[120:160]

,Spot Song,Spot Artist,Real Artist,Real Song,Song ID,fuzz_song,fuzz_artist,fuzz_partial_song
120,Freestyle,Lil Baby,Lil Baby,Freestyle,5BbdKBZO0TH0GhfxUfyhL9,100,100,100
121,Close Friends (Lil Baby & Gunna),Lil Baby,Lil Baby,Close Friends,5nzhL67GpF9szXWg5y509A,58,100,100
122,On Me,Lil Baby,Lil Baby,On Me,4Iedi94TIaB2GGb1nMB68v,100,100,100
123,The Bigger Picture,Lil Baby,Lil Baby,The Bigger Picture,44gRhRi2OhEf7moAUj6MD1,100,100,100
124,Emotionally Scarred,Lil Baby,Lil Baby,Emotionally Scarred,7ge7BHazYbVKi8qyZUX1Bm,100,100,100
125,Woah,Lil Baby,Lil Baby,Woah,598yNsG1JFrGd4n2kMoz7F,100,100,100
126,Sum 2 Prove,Lil Baby,Lil Baby,Sum 2 Prove,2u3EyuCqqV31woqGvO9LBz,100,100,100
127,Life Goes On,Lil Baby,Lil Baby,Life Goes On,2bSk87AVkCIIC3Bcligq1z,100,100,100
128,My Dawg,Lil Baby,Lil Baby,My Dawg,7CEJH5Ed7FBbXF6CqBQPlJ,100,100,100
129,Leaked,Lil Baby,Lil Baby,Leaked,0FIcYpNaqZaJnP3yDcaBN6,100,100,100


In [89]:
spot_data[160:]

,Spot Song,Spot Artist,Real Artist,Real Song,Song ID,fuzz_song,fuzz_artist,fuzz_partial_song
160,ROCKSTAR (feat. Roddy Ricch) - BLM REMIX,DaBaby,DaBaby,ROCKSTAR (BLM Remix),32hJ3k6vWJX4tGUx5txyJv,50,100,60
161,JUMP (feat. YoungBoy Never Broke Again),DaBaby,DaBaby,JUMP,0oT9ElXYSxvnOOagP9efDq,19,100,100
162,21,DaBaby,DaBaby,21,1AdXchAT6hBUm5d6y4nKjI,100,100,100
163,FIND MY WAY,DaBaby,DaBaby,FIND MY WAY,1gVCEnryJhkdQcuC0Kbvor,100,100,100
164,Walker Texas Ranger,DaBaby,DaBaby,Walker Texas Ranger,0eBOVXdnrVQ0rITgKXBAA7,100,100,100
...,...,...,...,...,...,...,...,...
985,Owe Me,Big Sean,Big Sean,Owe Me,1u0MOgWF1bbylVxpJG2ZjB,100,100,100
986,Research,Big Sean,Big Sean,Research,0nfkzX4y8O7Jxotd7bCu2C,100,100,100
987,Single Again,Big Sean,Big Sean,Single Again,7IP0lO9tHfNwBWZ9X6TJ84,100,100,100
988,Living Single,Big Sean,Big Sean,Living Single (Leak),36xxq7VDZZsGwARsn6Clo7,79,100,100


In [91]:
# New arrays to be added onto DataFrame for each song
test = spot_data
popularity, dance, energy, loudness, valence, tempo, instru, speech = ([] for i in range(8))

missing = 0
for i in range(0, len(test)):
    name = test.loc[i]['Spot Song'][:18]
    artist = test.loc[i]['Spot Artist']
    if name != 'NA':
        song_id = test.loc[i]['Song ID']
        print(f"Getting info for {name}")
        dance.append(spot.audio_features(song_id)[0]['danceability'])
        track = spot.search(q=f"{name} {artist}")
        pop = track['tracks']['items'][0]['popularity']
        popularity.append(pop)
        energy.append(spot.audio_features(song_id)[0]['energy'])
        loudness.append(spot.audio_features(song_id)[0]['loudness'])
        valence.append(spot.audio_features(song_id)[0]['valence'])
        tempo.append(spot.audio_features(song_id)[0]['tempo'])
        instru.append(spot.audio_features(song_id)[0]['instrumentalness'])
        speech.append(spot.audio_features(song_id)[0]['speechiness'])
    # Song is not found on Spotify; likely Soundcloud only and not on streaming services
    # Handle by adding 0 to song characteristics to filter out later
    else: 
        popularity.append(0)
        dance.append(0)
        energy.append(0)
        loudness.append(0)
        valence.append(0)
        tempo.append(0)
        instru.append(0)
        speech.append(0)
        # Prints song name if it is not here
        missing += 1
        print(str(missing) + ' Missing Info: ' + test.loc[i]['Real Song'])
# Add arrays of song characteristics into main DataFrame
main['popularity'] = popularity
main['dance'] = dance
main['energy'] = energy
main['loudness'] = loudness
main['valence'] = valence
main['tempo'] = tempo
main['instru'] = instru
main['speech'] = speech
# Length that has the proper characteristics

Getting info for Gods Plan
Getting info for In My Feelings
Getting info for Hotline Bling
Getting info for One Dance
Getting info for Hold On, Were Goin
Getting info for Know Yourself
Getting info for Back To Back
Getting info for All Me
Getting info for From Time
Getting info for Fake Love
Getting info for 0 To 100 / The Cat
Getting info for Pound Cake / Paris
Getting info for The Motto
Getting info for Started From the B
Getting info for Marvins Room
Getting info for HYFR (Hell Ya Fuck
Getting info for Passionfruit
Getting info for Nice For What
Getting info for Energy
Getting info for Childs Play
Getting info for Too Good
Getting info for Furthest Thing
Getting info for Jungle
Getting info for Over My Dead Body
Getting info for Worst Behavior
Getting info for Summer Sixteen
Getting info for The Language
Getting info for Girls Love Beyoncé
Getting info for How Bout Now
Getting info for Too Much
Getting info for SICKO MODE
Getting info for goosebumps
Getting info for BUTTERFLY EFFECT


Getting info for R.I.C.O. (feat. Dr
Getting info for Going Bad (feat. D
17 Missing Info: Wanna Know
Getting info for Amen (feat. Drake)
Getting info for Whats Free (feat. 
Getting info for Ooh Kill Em
Getting info for Dangerous (feat. J
Getting info for Litty (feat. Tory 
Getting info for Burn (feat. Big Se
Getting info for Believe It (feat. 
Getting info for Lord Knows (feat. 
Getting info for 1942 Flows
Getting info for Levels
Getting info for Bad for You (feat.
Getting info for Heaven or Hell
Getting info for Ima Boss (feat. T.
Getting info for Blue Notes
18 Missing Info: War Pain
Getting info for Chiraq
Getting info for Lil Nigga Snupe
Getting info for Jump Out the Face 
Getting info for Monster
Getting info for Offended (feat. Yo
Getting info for Traumatized
Getting info for Respect the Game
Getting info for Save Me
Getting info for 24/7 (feat. Ella M
Getting info for We Ball (feat. You
Getting info for Mercy
Getting info for Monster
Getting info for Father Stretch My 
Getting inf

Getting info for March Madness
Getting info for Where Ya At (feat.
Getting info for Comin Out Strong (
Getting info for F*ck Up Some Comma
Getting info for Codeine Crazy
Getting info for Thought It Was a D
Getting info for I Won (feat. Kanye
Getting info for Used to This (feat
Getting info for Trap Niggas
Getting info for Move That Dope (fe
Getting info for Stick Talk
Getting info for Mask Off (feat. Ke
Getting info for Throw Away
38 Missing Info: Jersey
Getting info for WAIT FOR U (feat. 
Getting info for Selfish (feat. Rih
Getting info for Purple Reign
Getting info for Draco
Getting info for Sh!t
Getting info for Wicked
Getting info for Honest
Getting info for Turn On The Lights
Getting info for 56 Nights
Getting info for Real Sisters
Getting info for Perkys Calling
Getting info for Karate Chop (Remix
Getting info for The Percocet & Str
Getting info for Best Friend
Getting info for The London (feat. 
Getting info for Check
39 Missing Info: Power
Getting info for With That (feat. D
Ge

Getting info for Jungle
Getting info for Timeless (feat. DJ
Getting info for D.T.B. (Interlude)
Getting info for My Shit
Getting info for Swervin (feat. 6ix
Getting info for Look Back at It
Getting info for No Promises
Getting info for Not a Regular Pers
Getting info for Fall in Love
Getting info for Friend Zone
Getting info for 99 Problems & Mess
52 Missing Info: Half On a Baby
Getting info for Numbers (feat. Rod
Getting info for Come Closer (feat.
Getting info for Me and My Guitar
Getting info for Say A
Getting info for Just Like Me (feat
53 Missing Info: Temporary
Getting info for Demons and Angels 
Getting info for Odee
Getting info for 4 Min Convo (Favor
Getting info for Reply (feat. Lil U
Getting info for Stalking You
Getting info for No Comparison
Getting info for Money over Everyth
Getting info for Beast Mode (feat. 
Getting info for Startender (feat. 
Getting info for Love Drugs and Sex
Getting info for Oh Okay (feat. You
Getting info for Sold Out Dates (fe
Getting info for Dr

In [93]:
main

,Song,Artist,PopIndex,PageViews,popularity,dance,energy,loudness,valence,tempo,instru,speech
0,Gods Plan,Drake,1,9184066,83,0.754,0.449,-9.211,0.357,77.169,0.000083,0.1090
1,In My Feelings,Drake,2,7444933,76,0.835,0.626,-5.833,0.350,91.030,0.000060,0.1250
2,Hotline Bling,Drake,3,6946884,78,0.891,0.628,-7.863,0.552,134.966,0.000190,0.0551
3,One Dance,Drake,4,6031277,85,0.792,0.625,-5.609,0.370,103.967,0.001800,0.0536
4,"Hold On, Were Going Home",Drake,5,5499301,72,0.777,0.414,-7.440,0.287,100.002,0.000027,0.0946
...,...,...,...,...,...,...,...,...,...,...,...,...
985,Owe Me,Big Sean,26,265435,46,0.620,0.609,-8.831,0.404,142.042,0.000000,0.1450
986,Research,Big Sean,27,258563,47,0.764,0.759,-4.494,0.762,143.037,0.000000,0.2440
987,Single Again,Big Sean,28,246273,56,0.635,0.677,-5.548,0.253,94.950,0.000000,0.0909
988,Living Single (Leak),Big Sean,29,236720,63,0.714,0.620,-5.045,0.730,80.109,0.000000,0.2960


In [94]:


os.chdir('/Users/garcgabe/Downloads/side work/')
main.to_csv("TotalArtistsTop30.csv")


